In [1]:
## import data manipulation packages
import pandas as pd
import numpy as np
from sklearn.neighbors import DistanceMetric
from math import radians

## import dataset "medium size" as variable 'city' and drop NaN
cities = pd.read_excel('data/worldcities.xlsx')
ct = cities.dropna(axis = 'rows', how = 'any')
## considero solo al di sopra dell'equatore
## ct = ct[ct.lat >= 0]

In [2]:
#add london Start (spostando lievemente la longitudine, così che sia più ad est di London)
London_st = ct.loc[(ct['city'] == 'London') & (ct['iso3'] == 'GBR')]
London_st['city']='London_st' 
London_st['lng'] = London_st['lng'] + 0.2

<ipython-input-2-b81702dafba4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  London_st['city']='London_st'
<ipython-input-2-b81702dafba4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  London_st['lng'] = London_st['lng'] + 0.2


In [3]:
## attacco la riga London_st al dataframe
ct = ct.append(London_st)

In [4]:
## resetto l'index, che sia progressivo
ct = ct.reset_index()


In [5]:
## concatenate iso2 and city to get unique id
ct['ID'] = ct['city'].map(str) + ct['iso2'].map(str)

In [6]:
## drop not usable columns
ct = ct.drop(['city_ascii', 'country', 'iso2', 'admin_name', 'capital', 'id'], axis = 1)

In [7]:
## droppo la colonna index duplicata
ct = ct.drop('index', axis = 1)

In [8]:
ct

,city,lat,lng,iso3,population,ID
0,Tokyo,35.6897,139.6922,JPN,3.797700e+07,TokyoJP
1,Jakarta,-6.2146,106.8451,IDN,3.454000e+07,JakartaID
2,Delhi,28.6600,77.2300,IND,2.961700e+07,DelhiIN
3,Mumbai,18.9667,72.8333,IND,2.335500e+07,MumbaiIN
4,Manila,14.5958,120.9772,PHL,2.308800e+07,ManilaPH
...,...,...,...,...,...,...
6618,Sayḩūt,15.2105,51.2454,YEM,1.890000e+02,SayḩūtYE
6619,Río Cuarto,-33.1230,-64.3478,ARG,1.582980e+02,Río CuartoAR
6620,La Esmeralda,3.1738,-65.5466,VEN,1.500000e+02,La EsmeraldaVE
6621,Hödrögö,48.9664,96.7833,MNG,1.000000e+01,HödrögöMN


In [9]:
# population - dove la popolazione è > di 200,000 abitanti aggiungo peso 2
pop = np.where(ct['population'] < 200000 , 0 , 2 )
#same state  - dove lo stato è diverso aggiungo peso 2
i = ct['iso3'].to_numpy()
st = (i[:, None ] != i) * 2

In [10]:
# direction - lavoro sulla longitudine e prendo le longitudini ad "est", normalizzando le long negative aggiungendo 150
## forse dobbiamo aggiungere 180? visto che dovrebbe essere su 360 totale?
dr_x = np.where(ct['lng']>= 0 , ct['lng'] , (ct['lng'] + 180) + 180)
x = dr_x

dr= (x[:, None ] < x)*1
rang= (x[: , None ] < x+60 )*1

In [11]:
dir_test = pd.DataFrame(dr*rang.T, columns = ct['ID'], index = ct['ID'])
dir_test

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,...,RodeoAR,GastreAR,TinogastaAR,TelsenAR,GreytownNI,SayḩūtYE,Río CuartoAR,La EsmeraldaVE,HödrögöMN,London_stGB
ID,,,,,,,,,,,,,,,,,,,,,
TokyoJP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
JakartaID,1,0,0,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
DelhiIN,0,1,0,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
MumbaiIN,0,1,1,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
ManilaPH,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SayḩūtYE,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Río CuartoAR,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
La EsmeraldaVE,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [12]:
# metto i giusti indici alle matrici 
direction = pd.DataFrame(dr*rang.T, columns = ct['ID'], index = ct['ID'])
same_state = pd.DataFrame(st, columns = ct['ID'], index = ct['ID'])
population = pd.DataFrame(pop , index = ct['ID'])

In [13]:
direction

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,...,RodeoAR,GastreAR,TinogastaAR,TelsenAR,GreytownNI,SayḩūtYE,Río CuartoAR,La EsmeraldaVE,HödrögöMN,London_stGB
ID,,,,,,,,,,,,,,,,,,,,,
TokyoJP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
JakartaID,1,0,0,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
DelhiIN,0,1,0,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
MumbaiIN,0,1,1,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
ManilaPH,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SayḩūtYE,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Río CuartoAR,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
La EsmeraldaVE,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [14]:
## coversion of coordinated in degrees to radians
ct['lat'] = np.radians(ct['lat'])
ct['lng'] = np.radians(ct['lng'])

In [15]:
## get the scipy metrics
dist = DistanceMetric.get_metric('haversine')

In [16]:
## prepare the array to get in input for the distance calculation
#non credo che serva 

#ct[['lat','lng']].to_numpy()

In [17]:
## pass pairwise function to get the distance / multiply for 6373 as earth sperical radius is 6373kms
#dist.pairwise(ct [['lat','lng']].to_numpy())*6373

In [18]:
#cerchiamo di avere meno dati 
D = np.where(  direction > 0  , dist.pairwise(ct [['lat','lng']].to_numpy())*6373 , 0   )
## create the distance matrix with cities in the indexes
distance = pd.DataFrame(D.T, columns = ct['ID'], index = ct['ID'])

In [19]:
distance

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,...,RodeoAR,GastreAR,TinogastaAR,TelsenAR,GreytownNI,SayḩūtYE,Río CuartoAR,La EsmeraldaVE,HödrögöMN,London_stGB
ID,,,,,,,,,,,,,,,,,,,,,
TokyoJP,0.0,5787.903765,0.000000,0.000000,2997.19434,1762.022389,0.0,1151.307887,0.000000,2903.800481,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,3763.889900,0.000000
JakartaID,0.0,0.000000,5011.267565,4662.287622,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,6573.440879,0.0,0.000000,6215.368372,0.000000
DelhiIN,0.0,0.000000,0.000000,1166.997459,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,3059.249501,0.0,0.000000,0.000000,0.000000
MumbaiIN,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2331.235749,0.0,0.000000,0.000000,0.000000
ManilaPH,0.0,2790.717691,4755.337989,5135.012933,0.00000,0.000000,0.0,0.000000,0.000000,1248.624550,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,4413.827801,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SayḩūtYE,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,6052.513438
Río CuartoAR,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,6915.216994,0.000000,...,557.012514,1103.133020,641.220957,1054.985097,5310.978147,0.000000,0.0,4039.257048,0.000000,0.000000
La EsmeraldaVE,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,4071.136107,0.000000,...,3733.495868,5067.878714,3481.586365,5069.283104,2179.599062,0.000000,0.0,0.000000,0.000000,0.000000


In [20]:
distance.loc['London_stGB'].sum()

5087.030191831854

In [21]:
from pandas import DataFrame

In [22]:
## popolare per criteri (più vicino +2, secondo più vicino +4, terzo più vicino +8, il resto 'no go')
dis = distance.T.replace(0, 0)
dis = dis.replace(dis.apply(lambda x: x[x > 0].min(axis=0)), 2)
dis = dis.replace(dis.apply(lambda x: x[x > 2].min(axis=0)), 4)
dis = dis.replace(dis.apply(lambda x: x[x > 4].min(axis=0)), 8)
dis = dis.where((dis <= 8), 0) 
dis

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,...,RodeoAR,GastreAR,TinogastaAR,TelsenAR,GreytownNI,SayḩūtYE,Río CuartoAR,La EsmeraldaVE,HödrögöMN,London_stGB
ID,,,,,,,,,,,,,,,,,,,,,
TokyoJP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JakartaID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DelhiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MumbaiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ManilaPH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SayḩūtYE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Río CuartoAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
La EsmeraldaVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
graph =((dis + same_state + pop.T) * dis/dis)
graph = graph.where((graph > 1), 0) 
    
graph

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,...,RodeoAR,GastreAR,TinogastaAR,TelsenAR,GreytownNI,SayḩūtYE,Río CuartoAR,La EsmeraldaVE,HödrögöMN,London_stGB
ID,,,,,,,,,,,,,,,,,,,,,
TokyoJP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JakartaID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DelhiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MumbaiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ManilaPH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SayḩūtYE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Río CuartoAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
La EsmeraldaVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
graph.to_csv('graph_marco')

In [25]:
graph.shape

(6623, 6623)

In [26]:
## un tentativo con networkX (mi prende il peso minore però----)

import networkx as nx

E = nx.from_pandas_adjacency(graph, create_using = nx.DiGraph)

nx.draw(E)

NetworkXError: Adjacency matrix not square: nx,ny=(6623, 6687)

In [ ]:
length, path = nx.single_source_dijkstra(D, source = 'London_stGB', target = 'London_stGB')
print(length)
print(path)